# Twitter data

## Copyright and Licensing

You are free to use or adapt this notebook for any purpose you'd like. However, please respect the [Simplified BSD License](https://github.com/ptwobrussell/Mining-the-Social-Web-2nd-Edition/blob/master/LICENSE.txt) that governs its use.

# Twitter API Access

Twitter implements OAuth 1.0A as its standard authentication mechanism, and in order to use it to make requests to Twitter's API, you'll need to go to https://dev.twitter.com/apps and create a sample application.

Choose any name for your application, write a description and use `http://google.com` for the website.

Under **Key and Access Tokens**, there are four primary identifiers you'll need to note for an OAuth 1.0A workflow: 
* consumer key, 
* consumer secret, 
* access token, and 
* access token secret (Click on Create Access Token to create those).

Note that you will need an ordinary Twitter account in order to login, create an app, and get these credentials.

The first time you execute the notebook, add all credentials so that you can save them in the `pkl` file, then you can remove the secret keys from the notebook because they will just be loaded from the `pkl` file.

The `pkl` file contains sensitive information that can be used to take control of your twitter acccount, **do not share it**.

In [21]:
import pickle
import os

In [22]:
if not os.path.exists('secret_twitter_credentials.pkl'):
    Twitter={}
    Twitter['Consumer Key'] = ''
    Twitter['Consumer Secret'] = ''
    Twitter['Access Token'] = ''
    Twitter['Access Token Secret'] = ''
    with open('secret_twitter_credentials.pkl','wb') as f:
        pickle.dump(Twitter, f)
else:
    Twitter=pickle.load(open('secret_twitter_credentials.pkl','rb'))

Install the `twitter` package to interface with the Twitter API

In [23]:
!pip install twitter

You are using pip version 9.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


## Example 1. Authorizing an application to access Twitter account data

In [24]:
import twitter

auth = twitter.oauth.OAuth(Twitter['Access Token'],
                           Twitter['Access Token Secret'],
                           Twitter['Consumer Key'],
                           Twitter['Consumer Secret'])

twitter_api = twitter.Twitter(auth=auth)

# Nothing to see by displaying twitter_api except that it's now a
# defined variable

print(twitter_api)

## Example 2. Retrieving trends

Twitter identifies locations using the Yahoo! Where On Earth ID.

The Yahoo! Where On Earth ID for the entire world is 1.
See https://dev.twitter.com/docs/api/1.1/get/trends/place and
http://developer.yahoo.com/geo/geoplanet/

look at the BOSS placefinder here: https://developer.yahoo.com/boss/placefinder/

In [25]:
WORLD_WOE_ID = 1
US_WOE_ID = 23424977

Look for the WOEID for [san-diego](http://woeid.rosselliot.co.nz/lookup/san%20diego%20%20ca)

You can change it to another location.

In [42]:
#help(twitter_api.trends.place)

In [34]:
LOCAL_WOE_ID=2487889

# Prefix ID with the underscore for query string parameterization.
# Without the underscore, the twitter package appends the ID value
# to the URL itself as a special case keyword argument.

world_trends = twitter_api.trends.place(_id=WORLD_WOE_ID)
us_trends = twitter_api.trends.place(_id=US_WOE_ID)
local_trends = twitter_api.trends.place(_id=LOCAL_WOE_ID)

In [54]:
type(world_trends)
vars(world_trends)
len(world_trends[0]['trends'])
world_trends[0]['trends'][0]

{'name': '#20MillionReasons',
 'promoted_content': None,
 'query': '%2320MillionReasons',
 'tweet_volume': 29913,
 'url': 'http://twitter.com/search?q=%2320MillionReasons'}

In [43]:
world_trends[:2]

[{'as_of': '2019-05-24T06:08:34Z',
  'created_at': '2019-05-24T06:01:53Z',
  'locations': [{'name': 'Worldwide', 'woeid': 1}],
  'trends': [{'name': '#20MillionReasons',
    'promoted_content': None,
    'query': '%2320MillionReasons',
    'tweet_volume': 29913,
    'url': 'http://twitter.com/search?q=%2320MillionReasons'},
   {'name': '#MeTeníaChato',
    'promoted_content': None,
    'query': '%23MeTen%C3%ADaChato',
    'tweet_volume': 34663,
    'url': 'http://twitter.com/search?q=%23MeTen%C3%ADaChato'},
   {'name': '#LoveBTS20M',
    'promoted_content': None,
    'query': '%23LoveBTS20M',
    'tweet_volume': 23221,
    'url': 'http://twitter.com/search?q=%23LoveBTS20M'},
   {'name': '#PhasesEP',
    'promoted_content': None,
    'query': '%23PhasesEP',
    'tweet_volume': 10011,
    'url': 'http://twitter.com/search?q=%23PhasesEP'},
   {'name': '#yağmur',
    'promoted_content': None,
    'query': '%23ya%C4%9Fmur',
    'tweet_volume': None,
    'url': 'http://twitter.com/search?q=%

In [56]:
trends=local_trends
print(type(trends))
print(list(trends[0].keys()))
#print(trends[0]['trends'])

<class 'twitter.api.TwitterListResponse'>
['trends', 'locations', 'created_at', 'as_of']


## Example 3. Displaying API responses as pretty-printed JSON

In [57]:
import json

print((json.dumps(us_trends[:2], indent=1)))

[
 {
  "trends": [
   {
    "tweet_volume": 10011,
    "query": "%23PhasesEP",
    "url": "http://twitter.com/search?q=%23PhasesEP",
    "name": "#PhasesEP",
    "promoted_content": null
   },
   {
    "tweet_volume": 29913,
    "query": "%2320MillionReasons",
    "url": "http://twitter.com/search?q=%2320MillionReasons",
    "name": "#20MillionReasons",
    "promoted_content": null
   },
   {
    "tweet_volume": 34734,
    "query": "%22Aaron+Rodgers%22",
    "url": "http://twitter.com/search?q=%22Aaron+Rodgers%22",
    "name": "Aaron Rodgers",
    "promoted_content": null
   },
   {
    "tweet_volume": 23221,
    "query": "%23LoveBTS20M",
    "url": "http://twitter.com/search?q=%23LoveBTS20M",
    "name": "#LoveBTS20M",
    "promoted_content": null
   },
   {
    "tweet_volume": null,
    "query": "%23TORvsMIL",
    "url": "http://twitter.com/search?q=%23TORvsMIL",
    "name": "#TORvsMIL",
    "promoted_content": null
   },
   {
    "tweet_volume": null,
    "query": "%23WeirdQuestions

## Example 4. Computing the intersection of two sets of trends

In [58]:
trends_set = {}
trends_set['world'] = set([trend['name'] 
                        for trend in world_trends[0]['trends']])

trends_set['us'] = set([trend['name'] 
                     for trend in us_trends[0]['trends']]) 

trends_set['san diego'] = set([trend['name'] 
                     for trend in local_trends[0]['trends']]) 

In [62]:
#trends_set['world']
#trends_set['us']
#trends_set['san diego']

In [63]:
for loc in ['world','us','san diego']:
    print(('-'*10,loc))
    print((','.join(trends_set[loc])))

('----------', 'world')
#HayırlıCumalar,#PhasesEP,#자캐는_체육대회에서_무슨_종목,#ExNaMTV,#HampaDelPeriodismo,#ChatpataElection,適応障害,#SaanNapupuntaAngPeraMo,#TORvsMIL,#ShowtimeMaCOOLay,ベンチマーク,Bucks in 7,#あなたの中身は何か,#LipsDontLie,#ทฤษฎีจีบเธอ,Gignac,#LaPiñaDeLaDiscordia,#THRBelumTerlihat,João Pedro,#يوم_الجمعه,#TNRejectsBJP,#PowerCoupleMulheres,#LoveBTS20M,#LosCruzados,#MeTeníaChato,ロスガル,#WeTheNorh,#BTS20M,Aaron Rodgers,#mykpopcareer,#yağmur,#RaptorsVsBucks,#mybtsenergy,#CongratulationsMheshimiwa,#KemenanganUntukSemua,#PactoDeSangre,#İtirazımızVar,#FikrimYok,#ياعلي,Fred VanVleet,#20MillionReasons,#ProtestoEdiyorum,ヴィエラ,#اختيار_التخصص_الجامعي,#PromediosEnTyCSports,#WeirdQuestionsForGoogle,#JumatBerkah,#SextaDetremuraSDV,#YoSoyAnimal,#مسابقه_المربع_نت١٩
('----------', 'us')
#SpaceX,#PhasesEP,Money Man,Dionne,#EscapeFromNewYork,#Elementary,#DoubleShotAtLove,Raptors in 6,Pusha T,#TORvsMIL,#StableGenius,Beast Coast,#SaveSocialBlade,Bucks in 7,#LipsDontLie,Steve Lacy,#CriticalRole,#SHADOWBRINGERS,#BTS20MPAR

In [64]:
print(( '='*10,'intersection of world and us'))
print((trends_set['world'].intersection(trends_set['us'])))

print(('='*10,'intersection of us and san-diego'))
print((trends_set['san diego'].intersection(trends_set['us'])))

('==========', 'intersection of world and us')
{'Aaron Rodgers', 'Fred VanVleet', '#20MillionReasons', '#WeirdQuestionsForGoogle', '#PhasesEP', '#RaptorsVsBucks', '#mybtsenergy', '#LoveBTS20M', '#TORvsMIL', 'Bucks in 7', '#LipsDontLie'}
('==========', 'intersection of us and san-diego')
{'#SpaceX', '#PhasesEP', 'Money Man', '#EscapeFromNewYork', '#DoubleShotAtLove', '#StableGenius', 'Pusha T', '#TORvsMIL', '#SaveSocialBlade', 'Beast Coast', '#LipsDontLie', 'Steve Lacy', '#CriticalRole', '#SHADOWBRINGERS', '#BTS20MPARTY', 'Gucci Mane', '#LoveBTS20M', 'Chris Webber', '#Falcon9', '#insteadofgoodbye', 'Aaron Rodgers', 'Lucas Giolito', '#HustleAndSoul', '#43VAHEARTLESS', 'The London', 'Van Vleet', '#4REAL4REAL', '#mybtsenergy', '#ProjectRunway', 'Mallory Edens', 'Ross Bjork', 'Marv Albert', '#starlink', '#UnlikelyReasonsToMoveOut', 'Fred VanVleet', '#20MillionReasons', '#WeirdQuestionsForGoogle', '#CrossMe', 'Viera', '#WhenIAmOutWithFriends', 'Krassensteins', 'Eyes Off You', 'Moneybagg'}


## Example 5. Collecting search results

Set the variable `q` to a trending topic, 
or anything else for that matter. The example query below
was a trending topic when this content was being developed
and is used throughout the remainder of this chapter

In [79]:
q = '#MTVAwards' 

number = 100
#number = 2

# See https://dev.twitter.com/docs/api/1.1/get/search/tweets

search_results = twitter_api.search.tweets(q=q, count=number)
print(search_results)
statuses = search_results['statuses']

{'statuses': [{'created_at': 'Fri May 24 08:44:29 +0000 2019', 'lang': 'en', 'in_reply_to_user_id': None, 'in_reply_to_status_id_str': None, 'possibly_sensitive': False, 'user': {'screen_name': 'MTVAfrica', 'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme14/bg.gif', 'listed_count': 155, 'lang': 'en', 'translator_type': 'regular', 'profile_image_url_https': 'https://pbs.twimg.com/profile_images/981129505246011392/TitPqE8r_normal.jpg', 'default_profile_image': False, 'profile_background_tile': True, 'statuses_count': 31611, 'profile_sidebar_fill_color': '000000', 'profile_use_background_image': True, 'profile_sidebar_border_color': '000000', 'created_at': 'Wed Jun 13 15:59:55 +0000 2012', 'verified': True, 'following': False, 'default_profile': False, 'protected': False, 'is_translation_enabled': False, 'profile_image_url': 'http://pbs.twimg.com/profile_images/981129505246011392/TitPqE8r_normal.jpg', 'notifications': False, 'favourites_count': 2543, 'time_zone': 

In [81]:
print(statuses[0].keys())
print(search_results.keys())
print(search_results['search_metadata'])

dict_keys(['created_at', 'lang', 'in_reply_to_user_id', 'in_reply_to_status_id_str', 'possibly_sensitive', 'user', 'metadata', 'in_reply_to_status_id', 'coordinates', 'in_reply_to_screen_name', 'retweet_count', 'favorited', 'retweeted', 'in_reply_to_user_id_str', 'id', 'favorite_count', 'truncated', 'entities', 'text', 'is_quote_status', 'id_str', 'place', 'source', 'contributors', 'geo'])
dict_keys(['statuses', 'search_metadata'])
{'since_id': 0, 'refresh_url': '?since_id=1131843396656672769&q=%23MTVAwards&include_entities=1', 'count': 100, 'since_id_str': '0', 'max_id_str': '1131843396656672769', 'completed_in': 0.118, 'max_id': 1131843396656672769, 'next_results': '?max_id=1131741330969960447&q=%23MTVAwards&count=100&include_entities=1', 'query': '%23MTVAwards'}


In [83]:
print(len(statuses))
#print(statuses)
statuses[0]['text']

70


"Who's got your vote for BEST FIGHT? 👊 Hit 'em with some love at https://t.co/UTD4ICNzno &amp; see who wins at the 2019… https://t.co/3Hfy3yl9NF"

Twitter often returns duplicate results, we can filter them out checking for duplicate texts:

In [84]:
all_text = []
filtered_statuses = []
for s in statuses:
    if not s["text"] in all_text:
        filtered_statuses.append(s)
        all_text.append(s["text"])
statuses = filtered_statuses     

In [85]:
len(statuses)

44

In [86]:
[s['text'] for s in search_results['statuses']]

["Who's got your vote for BEST FIGHT? 👊 Hit 'em with some love at https://t.co/UTD4ICNzno &amp; see who wins at the 2019… https://t.co/3Hfy3yl9NF",
 "RT @MTV: #MTVAwards performer @MartinGarrix's video for #SummerDays is  sweaty daydream: https://t.co/jA1VlsWJ35 https://t.co/4tAYwe0huG",
 "RT @MTVAwards: 🎬And the nominees for BEST Performance in a Movie are...🎬\n\nDon't forget to vote for the #MTVAwards at https://t.co/srdtfvy53…",
 'RT @MTVAwards: 💥Vote NOW  for BEST HERO at the 2019 #MTVAwards at https://t.co/srdtfvPGrL 💥\n\nWho are you rooting for? See if they win Monda…',
 '✨YOUR BEST DOCUMENTARY NOMINEES✨ Vote now for your fave to win at https://t.co/UTD4ICNzno ＋ watch the #MTVAwards on… https://t.co/1gOiyBXcgh',
 'RT @MTV: Throwing it back to when the @13ReasonsWhy cast presented the @Stranger_Things cast with their Golden Popcorn at the 2017 #MTVAwar…',
 'And your vote for Breakthrough Performance goes to ____? ⭐ Vote NOW for your pick at https://t.co/UTD4ICNzno ＋ tune… https://

In [87]:
# Show one sample search result by slicing the list...
print(json.dumps(statuses[0], indent=1))

{
 "created_at": "Fri May 24 08:44:29 +0000 2019",
 "lang": "en",
 "in_reply_to_user_id": null,
 "in_reply_to_status_id_str": null,
 "possibly_sensitive": false,
 "user": {
  "screen_name": "MTVAfrica",
  "profile_background_image_url": "http://abs.twimg.com/images/themes/theme14/bg.gif",
  "listed_count": 155,
  "lang": "en",
  "translator_type": "regular",
  "profile_image_url_https": "https://pbs.twimg.com/profile_images/981129505246011392/TitPqE8r_normal.jpg",
  "default_profile_image": false,
  "profile_background_tile": true,
  "statuses_count": 31611,
  "profile_sidebar_fill_color": "000000",
  "profile_use_background_image": true,
  "profile_sidebar_border_color": "000000",
  "created_at": "Wed Jun 13 15:59:55 +0000 2012",
  "verified": true,
  "following": false,
  "default_profile": false,
  "protected": false,
  "is_translation_enabled": false,
  "profile_image_url": "http://pbs.twimg.com/profile_images/981129505246011392/TitPqE8r_normal.jpg",
  "notifications": false,
  "fa

In [88]:
# The result of the list comprehension is a list with only one element that
# can be accessed by its index and set to the variable t
t = statuses[0]
#[ status for status in statuses 
#          if status['id'] == 316948241264549888 ][0]

# Explore the variable t to get familiarized with the data structure...

print(t['retweet_count'])
print(t['retweeted'])


0
False


In [93]:
t.keys()

dict_keys(['created_at', 'lang', 'in_reply_to_user_id', 'in_reply_to_status_id_str', 'possibly_sensitive', 'user', 'metadata', 'in_reply_to_status_id', 'coordinates', 'in_reply_to_screen_name', 'retweet_count', 'favorited', 'retweeted', 'in_reply_to_user_id_str', 'id', 'favorite_count', 'truncated', 'entities', 'text', 'is_quote_status', 'id_str', 'place', 'source', 'contributors', 'geo'])

## Example 6. Extracting text, screen names, and hashtags from tweets

In [94]:
status_texts = [ status['text'] 
                 for status in statuses ]

screen_names = [ user_mention['screen_name'] 
                 for status in statuses
                     for user_mention in status['entities']['user_mentions'] ]

hashtags = [ hashtag['text'] 
             for status in statuses
                 for hashtag in status['entities']['hashtags'] ]

# Compute a collection of all words from all tweets
words = [ w 
          for t in status_texts 
              for w in t.split() ]

In [95]:
# Explore the first 5 items for each...

print(json.dumps(status_texts[0:5], indent=1))
print(json.dumps(screen_names[0:5], indent=1)) 
print(json.dumps(hashtags[0:5], indent=1))
print(json.dumps(words[0:5], indent=1))

[
 "Who's got your vote for BEST FIGHT? \ud83d\udc4a Hit 'em with some love at https://t.co/UTD4ICNzno &amp; see who wins at the 2019\u2026 https://t.co/3Hfy3yl9NF",
 "RT @MTV: #MTVAwards performer @MartinGarrix's video for #SummerDays is  sweaty daydream: https://t.co/jA1VlsWJ35 https://t.co/4tAYwe0huG",
 "RT @MTVAwards: \ud83c\udfacAnd the nominees for BEST Performance in a Movie are...\ud83c\udfac\n\nDon't forget to vote for the #MTVAwards at https://t.co/srdtfvy53\u2026",
 "RT @MTVAwards: \ud83d\udca5Vote NOW  for BEST HERO at the 2019 #MTVAwards at https://t.co/srdtfvPGrL \ud83d\udca5\n\nWho are you rooting for? See if they win Monda\u2026",
 "\u2728YOUR BEST DOCUMENTARY NOMINEES\u2728 Vote now for your fave to win at https://t.co/UTD4ICNzno \uff0b watch the #MTVAwards on\u2026 https://t.co/1gOiyBXcgh"
]
[
 "MTV",
 "MartinGarrix",
 "MTVAwards",
 "MTVAwards",
 "MTV"
]
[
 "MTVAwards",
 "SummerDays",
 "MTVAwards",
 "MTVAwards",
 "MTVAwards"
]
[
 "Who's",
 "got",
 "your",
 "vote",
 "f

## Example 7. Creating a basic frequency distribution from the words in tweets

In [ ]:
from collections import Counter

for item in [words, screen_names, hashtags]:
    c = Counter(item)
    print(c.most_common()[:10]) # top 10
    print()

## Example 8. Create a prettyprint function to display tuples in a nice tabular format

In [ ]:
def prettyprint_counts(label, list_of_tuples):
    print("\n{:^20} | {:^6}".format(label, "Count"))
    print("*"*40)
    for k,v in list_of_tuples:
        print("{:20} | {:>6}".format(k,v))

In [ ]:
for label, data in (('Word', words), 
                    ('Screen Name', screen_names), 
                    ('Hashtag', hashtags)):
    
    c = Counter(data)
    prettyprint_counts(label, c.most_common()[:10])

## Example 9. Finding the most popular retweets

In [ ]:
retweets = [
            # Store out a tuple of these three values ...
            (status['retweet_count'], 
             status['retweeted_status']['user']['screen_name'],
             status['text'].replace("\n","\\")) 
            
            # ... for each status ...
            for status in statuses 
            
            # ... so long as the status meets this condition.
                if 'retweeted_status' in status
           ]

We can build another `prettyprint` function to print entire tweets with their retweet count.

We also want to split the text of the tweet in up to 3 lines, if needed.

In [ ]:
row_template = "{:^7} | {:^15} | {:50}"
def prettyprint_tweets(list_of_tuples):
    print()
    print(row_template.format("Count", "Screen Name", "Text"))
    print("*"*60)
    for count, screen_name, text in list_of_tuples:
        print(row_template.format(count, screen_name, text[:50]))
        if len(text) > 50:
            print(row_template.format("", "", text[50:100]))
            if len(text) > 100:
                print(row_template.format("", "", text[100:]))

In [ ]:
# Slice off the first 5 from the sorted results and display each item in the tuple

prettyprint_tweets(sorted(retweets, reverse=True)[:10])